In [243]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
import sklearn
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.externals import joblib
from keras import Sequential
from keras.layers import LSTM, Dropout, Dense
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing 
from xgboost import XGBRegressor, plot_importance
from matplotlib import pyplot

In [244]:
import os

data_path = '../input'
# data_path = 'data'
out_path = '../input'
submission_path = '../input'
ver = 6

In [245]:
def unreasonable_data(data):
    print("----------Reasonable of Data----------")
    print("Min Value:", data.min())
    print("Max Value:", data.max())
    print("Average Value:", data.mean())
    print("Center Point of Data:", data.median())
    print(data.describe())


In [246]:
def drop_duplicate(data, sub_set):
    print('Before drop shape:', data.shape)
    before = data.shape[0]
    data.drop_duplicates(sub_set, keep='first', inplace=True)
    data.reset_index(drop=True, inplace=True)
    print('After drop shape:', data.shape)
    after = data.shape[0]
    print('Total Duplicate:', before - after)

In [247]:
def pre_process_data_3():
    sales = pd.read_csv('%s/train.csv' % data_path, parse_dates=['date'], infer_datetime_format=True,dayfirst=True)
    val = pd.read_csv('%s/test.csv' % data_path)
    subset_train = ['date', 'date_block_num', 'shop_id', 'item_id', 'item_cnt_day']
    subset_test = ['date', 'date_block_num', 'shop_id', 'item_id']
    drop_duplicate(sales, sub_set=subset_train)
    drop_duplicate(val, sub_set=subset_test)

    # unreasonable_data(sales['item_cnt_day'])
    # unreasonable_data(sales['item_price'])

    median = sales[(sales.shop_id == 32) & (sales.item_id == 2973) & (sales.date_block_num == 4) & (
            sales.item_price > 0)].item_price.median()
    sales.loc[sales.item_price < 0, 'item_price'] = median
    sales['item_cnt_day'] = sales['item_cnt_day'].clip(0, 1000)
    # sales['item_cnt_day'] = sales['item_cnt_day'].clip(0, 20)
    sales['item_price'] = sales['item_price'].clip(0, 300000)

    # =======================
    # From https://www.kaggle.com/dlarionov/feature-engineering-xgb/notebook
    # Якутск Орджоникидзе, 56
    sales.loc[sales.shop_id == 0, 'shop_id'] = 57
    val.loc[val.shop_id == 0, 'shop_id'] = 57
    # Якутск ТЦ "Центральный"
    sales.loc[sales.shop_id == 1, 'shop_id'] = 58
    val.loc[val.shop_id == 1, 'shop_id'] = 58
    # Жуковский ул. Чкалова 39м²
    sales.loc[sales.shop_id == 10, 'shop_id'] = 11
    val.loc[val.shop_id == 10, 'shop_id'] = 11
    # =======================
    for f in sales.columns: 
        if sales[f].dtype=='object': 
            lbl = preprocessing.LabelEncoder() 
            lbl.fit(list(train[f].values)) 
            sales[f] = lbl.transform(list(sales[f].values))

    for f in val.columns: 
        if val[f].dtype=='object': 
            lbl = preprocessing.LabelEncoder() 
            lbl.fit(list(val[f].values)) 
            val[f] = lbl.transform(list(val[f].values))

    sales.fillna((-999), inplace=True) 
    val.fillna((-999), inplace=True)


  

    # Rearrange the raw data to be monthly sales by item-shop
    #sales.date.apply(lambda x: x.strftime('%Y-%m'))
    df = sales
    df['day']=df['date'].apply(lambda x: x.strftime('%d'))
    df['day']=df['day'].astype('int64')
    df['month']=df['date'].apply(lambda x: x.strftime('%m'))
    df['month']=df['month'].astype('int64')
    df['year']=df['date'].apply(lambda x: x.strftime('%Y'))
    df['year']=df['year'].astype('int64')
    df = df[['day','month','year','item_id', 'shop_id','item_price','item_cnt_day']]
#     df = df.pivot_table(index=['item_id', 'shop_id'], columns='date', values='item_cnt_day', fill_value=0).reset_index()
#     count=df.iloc[:,2:]
#     df['total']=count.sum(axis=1)
    data=df
    #data = pd.merge(val, df, on=['item_id', 'shop_id'], how='left').fillna(0)
    data['item_id'] = np.log1p(data['item_id'])
    return data

In [248]:
#  df = sales
# df['day']=df['date'].apply(lambda x: x.strftime('%d'))
# df['day']=df['day'].astype('int64')
# df = df[['day','date_block_num','item_id', 'shop_id','item_price','item_cnt_day']]
# #     df = df.pivot_table(index=['item_id', 'shop_id'], columns='date', values='item_cnt_day', fill_value=0).reset_index()
# #     count=df.iloc[:,2:]
# #     df['total']=count.sum(axis=1)
# data=df
# #data = pd.merge(val, df, on=['item_id', 'shop_id'], how='left').fillna(0)
# data['item_id'] = np.log1p(data['item_id'])
# df.head(10)

In [249]:
# df['day']=df['date'].apply(lambda x: x.strftime('%d'))
# df['day']=df['day'].astype('int64')
# df.head(20)

In [250]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
day           10000 non-null int64
month         10000 non-null int64
year          10000 non-null int64
item_id       10000 non-null float64
shop_id       10000 non-null int64
item_price    10000 non-null float64
dtypes: float64(2), int64(4)
memory usage: 468.8 KB


In [251]:

# df = df.pivot_table(index=['item_id', 'shop_id'], columns='date', values='item_cnt_day', fill_value=0).reset_index()
# df.head(30)

In [252]:
# count=df.iloc[:,2:]
# df['total']=count.sum(axis=1)


In [253]:
# df.head()

In [254]:
# val = pd.read_csv('%s/test.csv' % data_path)
# data = pd.merge(val, df, on=['item_id', 'shop_id'], how='left').fillna(0)


In [255]:
# data['item_id'] = np.log1p(data['item_id'])
# data.head(30)


In [256]:
def rmse(predictions, targets):
    return np.sqrt(np.mean((predictions - targets) ** 2))


In [257]:
def train_model(x_train, y_train):
    reg = ExtraTreesRegressor(n_estimators=512, max_depth=20,
                                       random_state=50)
    reg.fit(x_train, y_train)
    y_pre = reg.predict(x_train)
    score = np.sqrt(mean_squared_error(y_train, y_pre))
    print('RMSE cliped:', np.sqrt(mean_squared_error(y_train.clip(0., 20.), y_pre.clip(0., 20.))))
    return reg

In [258]:
def light_gbm_model(x_train, y_train):
    lgb_params = {
        'feature_fraction': 1,
        'metric': 'rmse',
        'min_data_in_leaf': 16,
        'bagging_fraction': 0.85,
        'learning_rate': 0.03,
        'objective': 'mse',
        'bagging_seed': 2 ** 7,
        'num_leaves': 32,
        'bagging_freq': 3,
        'verbose': 0
    }
    estimator = lgb.train(lgb_params, lgb.Dataset(x_train, label=y_train), 300)
    y_pre = estimator.predict(x_train)
    print('RMSE cliped:', np.sqrt(mean_squared_error(y_train.clip(0., 20.), y_pre.clip(0., 20.))))
    return estimator

In [259]:
def linear_model(x_train, y_train):
    lr = LinearRegression()
    lr.fit(x_train, y_train)
    y_pre = lr.predict(x_train)
    print('RMSE cliped:', np.sqrt(mean_squared_error(y_train.clip(0., 20.), y_pre.clip(0., 20.))))
    return lr

In [260]:
def xgb_model(x_train, y_train, x_train_val, y_train_val):
    model = XGBRegressor(
        max_depth=8,
        n_estimators=1000,
        min_child_weight=300,
        colsample_bytree=0.9,
        subsample=0.9,
        eta=0.15,
        seed=42)
    model.fit(
        x_train,
        y_train,
        eval_metric="rmse",
        eval_set=[(x_train, y_train), (x_train_val, y_train_val)],
        verbose=True,
        early_stopping_rounds=10)
    y_pre = model.predict(x_train)
    print('RMSE cliped:', np.sqrt(mean_squared_error(y_train.clip(0., 20.), y_pre.clip(0., 20.))))
    plot_importance(model)
    pyplot.show()
    return model

In [261]:
def pre_data(data_type, reg, x_test):
    if reg is None:
        reg = joblib.load('%s/%s_model_weight.model' % (out_path, data_type))
    y_pre = reg.predict(x_test)
    return y_pre

In [262]:
test = pre_process_data_3()
test_date_info = test




Before drop shape: (2925849, 7)
After drop shape: (2925827, 7)
Total Duplicate: 22
Before drop shape: (10000, 6)
After drop shape: (10000, 6)
Total Duplicate: 0


C:\Users\Siddu\Miniconda3\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [263]:
# sales = pd.read_csv('%s/train.csv' % data_path, parse_dates=['date'], infer_datetime_format=True,dayfirst=True)

y_train_normal = test_date_info['item_cnt_day']
x_train_normal = test_date_info.drop(labels=['item_cnt_day'], axis=1)


In [268]:
#x_train_normal.columns = np.append(['shop_id', 'item_id'],np.arange(0, 36, 1))
x_train_val = x_train_normal[-100:]
y_train_val = y_train_normal[-100:]

,day,month,year,item_id,shop_id,item_price
2925727,16,1,2014,9.509407,56,12990.0
2925728,30,5,2015,8.806274,18,26990.0
2925729,30,3,2015,8.669399,6,1099.0
2925730,11,9,2015,8.525360,38,1249.5
2925731,8,6,2014,9.668524,56,199.0


In [266]:
#xgb_model = xgb_model(x_train_normal[:-100], y_train_normal[:-100], x_train_val, y_train_val)
#linear_model = linear_model(x_train_normal, y_train_normal)
#light_gbm_model = light_gbm_model(x_train_normal, y_train_normal)
# normal_model = train_model(x_train_normal, y_train_normal)

RMSE cliped: 1.135070594287477


In [240]:
parser = lambda date: pd.to_datetime(date, format='%d.%m.%Y')

val = pd.read_csv('%s/test.csv' % data_path, parse_dates=['date'], date_parser=parser)
subset_test = ['date', 'date_block_num', 'shop_id', 'item_id']
drop_duplicate(val, sub_set=subset_test)

# unreasonable_data(sales['item_cnt_day'])
# unreasonable_data(sales['item_price'])

val.loc[val.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
val.loc[val.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
val.loc[val.shop_id == 10, 'shop_id'] = 11
# =======================

# for f in val.columns: 
#     if val[f].dtype=='object': 
#         lbl = preprocessing.LabelEncoder() 
#         lbl.fit(list(val[f].values)) 
#         val[f] = lbl.transform(list(val[f].values))

sales.fillna((-999), inplace=True) 
val.fillna((-999), inplace=True)



df = val
val.info()
val['day']=val['date'].apply(lambda x:x.strftime('%d'))
df['day']=df['day'].astype('int64')
df['month']=df['date'].apply(lambda x: x.strftime('%m'))
df['month']=df['month'].astype('int64')
df['year']=df['date'].apply(lambda x: x.strftime('%Y'))
df['year']=df['year'].astype('int64')
df = df[['day','month','year','item_id', 'shop_id','item_price']]
data=df
#data = pd.merge(val, df, on=['item_id', 'shop_id'], how='left').fillna(0)
data['item_id'] = np.log1p(data['item_id'])
test_x = data
# test_x.columns = np.append(['shop_id', 'item_id'],np.arange(0, 36, 1))
# test_y_1 = pre_data('normal', normal_model, test_x)
test_y_2 = pre_data('light_gbm', light_gbm_model, test_x)
# test_y_3 = pre_data('linear', linear_model, test_x)
#test_y_4 = pre_data('xgb', xgb_model, test_x)
test_y = test_y_2
val.head()

Before drop shape: (10000, 6)
After drop shape: (10000, 6)
Total Duplicate: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
date              10000 non-null datetime64[ns]
date_block_num    10000 non-null int64
shop_id           10000 non-null int64
item_id           10000 non-null int64
item_price        10000 non-null float64
ID                10000 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 468.8 KB


C:\Users\Siddu\Miniconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,date,date_block_num,shop_id,item_id,item_price,ID,day,month,year
0,2015-02-06,25,31,11208,699.0,2472142,6,2,2015
1,2013-11-21,10,31,19525,149.0,1044690,21,11,2013
2,2015-01-03,24,42,16169,299.0,2367559,3,1,2015
3,2014-07-18,18,42,2737,199.0,1793501,18,7,2014
4,2014-04-13,15,2,15229,1199.0,1585639,13,4,2014


In [241]:
test_y.shape

(10000,)

In [242]:
val['item_cnt_day'] = test_y
val[['ID', 'item_cnt_day']].to_csv('submission_xgboost.csv', index=False)